 This file extracts frames from video that has an annotated bounding box and creating a new json file as 
 new labels for each frame. 

### inputs files:
1. data_interim_annotate_video.xlsx: providing all informations needed to put into label files except the bounding box
2. video.json: exported from supervisely containing inforamtion of bounding box of each frame
3. original video file

### outputs:
- Folder 1 contains:
    label.json: containing all relatively infos from data_interim_annotate_video.xlsx + bounding box
- Folder 2 contains:
    frame.jpg: individual frame with a bounding box annotation from video 


## RUN ONLY ONCE

In [63]:
# uncomment below to install objectpath module in the anaconda environment
#! python3 -m pip install objectpath

You should consider upgrading via the '/Users/xinwen/opt/anaconda3/bin/python3 -m pip install --upgrade pip' command.


In [294]:
# create new label and new image folder for the FIRST TIME ONLY
try:
    os.makedirs(f'{rootdir}/new_label')
    os.makedirs(f'{rootdir}/new_image')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
# import libraries
import cv2
import os
import json
import pandas as pd
import re
import objectpath
import numpy as np

In [295]:
# define universe variable
rootdir = "/Users/xinwen/Desktop/video"
video_summary = "data_interim_annotate_video.xlsx"


def read_excel(summary,video):
    ''' input: summary: video summary excel sheet
               video: video file name
        output: one row dataframe containing the video's info 
    '''
    return pd.read_excel(rootdir+'/'+summary).loc[df.file_name_new == video]


def get_info_from_json(path):
    '''input: path: video label file path (.json)
       output: a list of frame number and a matching list of bounding boxes for each frame
               (some frame might have multiple bounding box annotations)
    '''
    with open(path) as f:
        data = json.load(f)
        label_info = objectpath.Tree(data)
        frame_num = list(label_info.execute('$.frames.index'))
        frame_info = list(label_info.execute('$.frames.*exterior'))
        bbox = []
        for x in frame_info: # each frame
            bbox.append([y['geometry']['points']['exterior'] for y in x['figures']])  
        return frame_num,bbox

def create_new_json(bbox,summary,video):
    '''input: bbox: bbox from get_info_from_json(path)
              summary: result from read_excel()
              video: video file name
       output: create new json file for each annotated frame
    '''
    for i in range(0,len(bbox)): 
        data = {}
        data['bbox'] = bbox[i]
        for j in list(summary):
            if type(summary.iloc[0][j])in (np.float64,np.int64):
                if np.isnan(summary.iloc[0][j]) :
                    data[f'{j}'] = ""
                else:
                    data[f'{j}'] = int(summary.iloc[0][j])
            else:
                data[f'{j}'] = summary.iloc[0][j]
        with open(rootdir+f'/new_label/{video}_{i}.json','w') as outfile:
            json.dump(data,outfile)



In [298]:
def cropFrame(video,frame_list):
    """ Input: video: video file name
               frame_list: frame_num from get_info_from_json(path)
        Output: crop frames according to frame_list for each video
    """
    vidcap = cv2.VideoCapture(video)
    for i in frame_list:
        vidcap.set(1,i)
        hasFrames,image = vidcap.read()
        sucess = hasFrames
        if sucess:
            cv2.imwrite(f'new_image/{video}_{i}.jpg', image)

            
def generate_image_from_video():
    """ output: for each video in the folder
                    1.get the corresponding frame number and bounding box
                    2.extract the frames from the video
                    3.get all infos of the video from video_summary xlsx
                    4.create new json label for each extracted frames
    """
    for filename in os.listdir(rootdir):
        if filename.endswith(".mp4"):
            fn,bb = get_info_from_json(f"{rootdir}/{filename}.json") #1
            cropFrame(filename,fn)#2
            attributes = read_excel(video_summary,filename)#3
            create_new_json(bb,attributes,filename)#4
            


In [301]:
# crop frames
generate_image_from_video() 